In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('model_dataset.csv')

In [3]:
data.columns

Index(['PTNR_REGIONAL_MASTER_CD', 'YEARWEEK', 'TOTAL_SALES'], dtype='object')

In [4]:
data = data[(data['TOTAL_SALES'] > 0) & (~data['TOTAL_SALES'].isna())]


In [5]:
data['PTNR_REGIONAL_MASTER_CD'] = data['PTNR_REGIONAL_MASTER_CD'].astype(str)

In [6]:
data.head()

,PTNR_REGIONAL_MASTER_CD,YEARWEEK,TOTAL_SALES
0,0.0,2019-12-02,28994.45
1,0.0,2019-12-09,127699.87
2,0.0,2019-12-16,102609.00
3,0.0,2020-06-01,37745.66
4,0.0,2020-06-08,72885.09


In [7]:
data['YEARWEEK'] = data['YEARWEEK'].astype(str)
data['YEARWEEK'] = pd.to_datetime(data['YEARWEEK'].str[:4] + '-' + data['YEARWEEK'].str[-2:] + '-1', format='%Y-%U-%w')

In [8]:
data.head(20)

,PTNR_REGIONAL_MASTER_CD,YEARWEEK,TOTAL_SALES
0,0.0,2019-01-14,2.899445e+04
1,0.0,2019-03-04,1.276999e+05
2,0.0,2019-04-22,1.026090e+05
3,0.0,2020-01-06,3.774566e+04
4,0.0,2020-02-24,7.288509e+04
5,0.0,2020-04-13,5.731133e+04
6,0.0,2020-06-01,3.557325e+04
7,0.0,2020-07-20,2.096550e+04
8,0.0,2020-02-10,1.066909e+04
9,0.0,2020-03-30,2.495384e+04


In [9]:
data.dtypes

PTNR_REGIONAL_MASTER_CD            object
YEARWEEK                   datetime64[ns]
TOTAL_SALES                       float64
dtype: object

In [10]:
grouped_data = data.groupby('PTNR_REGIONAL_MASTER_CD')

In [11]:
grouped_data.head(100)

,PTNR_REGIONAL_MASTER_CD,YEARWEEK,TOTAL_SALES
0,0.0,2019-01-14,28994.45
1,0.0,2019-03-04,127699.87
2,0.0,2019-04-22,102609.00
3,0.0,2020-01-06,37745.66
4,0.0,2020-02-24,72885.09
...,...,...,...
16367,50001145.0,2024-02-05,96965.71
16368,50001145.0,2024-03-25,89132.97
16369,50001145.0,2024-05-13,113202.00
16370,50001145.0,2024-07-01,129128.79


In [12]:
order = (1, 1, 1)
seasonal_order = (1, 1, 1, 52)

In [13]:
forecasts = {}

In [14]:
import statsmodels.api as sm 
from sklearn.model_selection import train_test_split

In [15]:
start_date = '2024-04-01'
end_date = '2024-06-30'
forecast_dates = pd.date_range(start=start_date, end=end_date, freq='W-MON')  # Assuming weekly forecasts on Mondays


forecast_df = pd.DataFrame(index=forecast_dates)


In [16]:
forecast_data = []

In [17]:
for group_name, group_data in grouped_data:
    print(f"Processing PTNR_REGIONAL_MASTER_CD: {group_name}")
    
    
    if len(group_data) < 2:
        print(f"Skipping PTNR_REGIONAL_MASTER_CD: {group_name} due to insufficient data for splitting")
        continue
    

    train_data, test_data = train_test_split(group_data, test_size=0.2, shuffle=False)  
    

    model = sm.tsa.statespace.SARIMAX(train_data['TOTAL_SALES'], order=order, seasonal_order=seasonal_order, enforce_stationarity=False)
    fitted_model = model.fit(disp=-1)
    

    forecast = fitted_model.forecast(steps=len(forecast_dates))
    

    forecasts[group_name] = forecast

Processing PTNR_REGIONAL_MASTER_CD: 0.0


/Users/harshitgupta/Desktop/vs /.venv/lib/python3.12/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'


Processing PTNR_REGIONAL_MASTER_CD: 50000001.0


/Users/harshitgupta/Desktop/vs /.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/harshitgupta/Desktop/vs /.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/harshitgupta/Desktop/vs /.venv/lib/python3.12/site-packages/statsmodels/tsa/statespace/sarimax.py:1009: UserWarning: Non-invertible starting seasonal moving average Using zeros as starting parameters.
  warn('Non-invertible starting seasonal moving average'
/Users/harshitgupta/Desktop/vs /.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/harshitgu

Processing PTNR_REGIONAL_MASTER_CD: 50000002.0


...

/Users/harshitgupta/Desktop/vs /.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/harshitgupta/Desktop/vs /.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/Users/harshitgupta/Desktop/vs /.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/harshitgupta/Desktop/vs /.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(da

Processing PTNR_REGIONAL_MASTER_CD: 50001136.0


/Users/harshitgupta/Desktop/vs /.venv/lib/python3.12/site-packages/statsmodels/tsa/statespace/sarimax.py:1009: UserWarning: Non-invertible starting seasonal moving average Using zeros as starting parameters.
  warn('Non-invertible starting seasonal moving average'
/Users/harshitgupta/Desktop/vs /.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/harshitgupta/Desktop/vs /.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/Users/harshitgupta/Desktop/vs /.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g

Processing PTNR_REGIONAL_MASTER_CD: 50001145.0


/Users/harshitgupta/Desktop/vs /.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/harshitgupta/Desktop/vs /.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [18]:
forecast_df = pd.DataFrame(forecasts)


print(forecast_df)

     0.0     50000001.0  50000002.0  50000004.0  50000006.0  50000007.0  \
24   NaN            NaN         NaN         NaN         NaN         NaN   
25   NaN            NaN         NaN         NaN         NaN         NaN   
26   NaN            NaN         NaN         NaN         NaN         NaN   
27   NaN            NaN         NaN         NaN         NaN         NaN   
28   NaN            NaN         NaN         NaN         NaN         NaN   
..   ...            ...         ...         ...         ...         ...   
268  NaN  265972.685281         NaN         NaN         NaN         NaN   
269  NaN  271864.952648         NaN         NaN         NaN         NaN   
270  NaN  270742.231664         NaN         NaN         NaN         NaN   
271  NaN            NaN         NaN         NaN         NaN         NaN   
272  NaN            NaN         NaN         NaN         NaN         NaN   

       50000009.0  50000010.0  50000011.0  50000014.0  ...  50000731.0  \
24            NaN        

In [19]:
evaluation_metrics = {}

In [20]:
import numpy as np

In [21]:
for group_name, group_data in grouped_data:
    print(f"Evaluating PTNR_REGIONAL_MASTER_CD: {group_name}")

    if len(group_data) < 2:
        print(f"Skipping PTNR_REGIONAL_MASTER_CD: {group_name} due to insufficient data for evaluation")
        continue
    

    train_data, test_data = train_test_split(group_data, test_size=0.2, shuffle=False)  
    
    # Check if there is data in the test set
    if len(test_data) == 0:
        print(f"No test data available for PTNR_REGIONAL_MASTER_CD: {group_name}. Skipping evaluation.")
        continue
    

    forecast_values = forecast_df[group_name]
  
    actual_values = test_data['TOTAL_SALES']
    

    absolute_errors = np.abs(forecast_values - actual_values)
    mean_absolute_error = absolute_errors.mean()
    mean_absolute_percentage_error = (absolute_errors / actual_values).mean() * 100
    

    evaluation_metrics[group_name] = {
        'Mean Absolute Error': mean_absolute_error,
        'Mean Absolute Percentage Error (%)': mean_absolute_percentage_error
    }

Evaluating PTNR_REGIONAL_MASTER_CD: 0.0
Evaluating PTNR_REGIONAL_MASTER_CD: 50000001.0
Evaluating PTNR_REGIONAL_MASTER_CD: 50000002.0
Evaluating PTNR_REGIONAL_MASTER_CD: 50000004.0
Evaluating PTNR_REGIONAL_MASTER_CD: 50000006.0
Evaluating PTNR_REGIONAL_MASTER_CD: 50000007.0
Evaluating PTNR_REGIONAL_MASTER_CD: 50000009.0
Evaluating PTNR_REGIONAL_MASTER_CD: 50000010.0
Evaluating PTNR_REGIONAL_MASTER_CD: 50000011.0
Evaluating PTNR_REGIONAL_MASTER_CD: 50000014.0
Evaluating PTNR_REGIONAL_MASTER_CD: 50000020.0
Evaluating PTNR_REGIONAL_MASTER_CD: 50000022.0
Evaluating PTNR_REGIONAL_MASTER_CD: 50000024.0
Evaluating PTNR_REGIONAL_MASTER_CD: 50000027.0
Evaluating PTNR_REGIONAL_MASTER_CD: 50000029.0
Evaluating PTNR_REGIONAL_MASTER_CD: 50000031.0
Evaluating PTNR_REGIONAL_MASTER_CD: 50000033.0
Evaluating PTNR_REGIONAL_MASTER_CD: 50000035.0
Evaluating PTNR_REGIONAL_MASTER_CD: 50000041.0
Evaluating PTNR_REGIONAL_MASTER_CD: 50000045.0
Evaluating PTNR_REGIONAL_MASTER_CD: 50000047.0
Evaluating PTNR_REGI

In [22]:
evaluation_df = pd.DataFrame(evaluation_metrics).T
print(evaluation_df)

            Mean Absolute Error  Mean Absolute Percentage Error (%)
0.0                2.392489e+06                           25.845171
50000001.0                  NaN                                 NaN
50000002.0                  NaN                                 NaN
50000004.0                  NaN                                 NaN
50000006.0                  NaN                                 NaN
...                         ...                                 ...
50001007.0                  NaN                                 NaN
50001066.0                  NaN                                 NaN
50001126.0                  NaN                                 NaN
50001136.0                  NaN                                 NaN
50001145.0                  NaN                                 NaN

[69 rows x 2 columns]
